In [1]:
from werkzeug.wrappers import Request, Response
import flask
from flask import Flask, jsonify
import joblib
import logging
import pandas as pd
import io
import numpy as np
import shap
import re

# NOTE: *not* for production use
# just testing simple API serving scenarios in this LAB
# -won't scale properly
# -needs much more security checking due to file handling, pickle and dynamic loading

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = Flask(__name__)

model_cache={}

@app.route("/")
def hello():
    return "Hello World!"

# Don't allow anything beside words/numbers _ and -, avoid traverse directories
def get_valid_modelid(model_id):
     return re.sub(r"[^a-zA-Z0-9_-]","_",model_id)
   

def get_model(model_id):
    
    model_id=get_valid_modelid(model_id)
    
    model_info=model_cache.get(model_id)
    if model_info is None:
        logger.info("Loading model %s..."%(model_id))
        
        model_pipeline=joblib.load(f"../../models/{model_id}.pkl")
        
        print("Loaded:",model_pipeline)

        model_info={"model_id":model_id,"model_pipeline":model_pipeline}

        model_cache[model_id]=model_info  
    return model_info

@app.route("/info/<model_id>/")
def info(model_id,methods=["GET"]):
    
    model_id=get_valid_modelid(model_id)
    
    model_info=get_model(model_id)
    
    return(str(model_info))

@app.route("/predict/<model_id>/",methods=["POST"])
def predict(model_id):
    
    model_id=get_valid_modelid(model_id)
    
    model_info=get_model(model_id)

    logger.info(flask.request.content_type)
    
    # Convert from CSV to pandas
    if flask.request.content_type.lower().endswith("csv"):
        print("reading csv...")
        data = flask.request.data.decode('utf-8')
        s = io.StringIO(data)
        data = pd.read_csv(s)
    elif flask.request.content_type == 'application/json':
        print("reading json...")
        print(flask.request.json)
        print("json end...")
        data=pd.DataFrame(flask.request.json)
    else:
        return flask.Response(response='This predictor only supports CSV or JSON data', status=415, mimetype='text/plain')
    
    print("Shape:",data.shape)
    print(data.head(5))
    
    # Get pipeline
    model_pipeline=model_info["model_pipeline"]

    # Predict
    y_hat=model_pipeline.predict_proba(data)
    
    df_out=pd.DataFrame(y_hat,index=data.index)
    
    return(df_out.to_json(orient="records"))



In [2]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9001, app)

INFO:werkzeug: * Running on http://localhost:9001/ (Press CTRL+C to quit)
INFO:__main__:Loading model pipeline_demo_best_v1...


Loaded: Pipeline(memory=None,
     steps=[('prep', PrepPipeline(add_missing_indicators=False, copy=True, impute_age=False,
       impute_cabin=True, notes='v1-grid search', train_filter='')), ('featurize', DataFrameMapper(default=False, df_out=True,
        features=[(['PassengerId'], Imputer(axis=0, copy=True, missing_values='NaN', ...stimators=50, n_jobs=-1,
            oob_score=False, random_state=1, verbose=0, warm_start=False))])


INFO:__main__:application/json


reading json...
[{'PassengerId': 892, 'Pclass': 3, 'Name': 'Kelly, Mr. James', 'Sex': 'male', 'Age': 34.5, 'SibSp': 0, 'Parch': 0, 'Ticket': '330911', 'Fare': 7.8292, 'Cabin': None, 'Embarked': 'Q'}, {'PassengerId': 893, 'Pclass': 3, 'Name': 'Wilkes, Mrs. James (Ellen Needs)', 'Sex': 'female', 'Age': 47.0, 'SibSp': 1, 'Parch': 0, 'Ticket': '363272', 'Fare': 7.0, 'Cabin': None, 'Embarked': 'S'}, {'PassengerId': 894, 'Pclass': 2, 'Name': 'Myles, Mr. Thomas Francis', 'Sex': 'male', 'Age': 62.0, 'SibSp': 0, 'Parch': 0, 'Ticket': '240276', 'Fare': 9.6875, 'Cabin': None, 'Embarked': 'Q'}, {'PassengerId': 895, 'Pclass': 3, 'Name': 'Wirz, Mr. Albert', 'Sex': 'male', 'Age': 27.0, 'SibSp': 0, 'Parch': 0, 'Ticket': '315154', 'Fare': 8.6625, 'Cabin': None, 'Embarked': 'S'}, {'PassengerId': 896, 'Pclass': 3, 'Name': 'Hirvonen, Mrs. Alexander (Helga E Lindqvist)', 'Sex': 'female', 'Age': 22.0, 'SibSp': 1, 'Parch': 1, 'Ticket': '3101298', 'Fare': 12.2875, 'Cabin': None, 'Embarked': 'S'}]
json end...


INFO:werkzeug:127.0.0.1 - - [12/Sep/2018 16:50:56] "POST /predict/pipeline_demo_best_v1/ HTTP/1.1" 200 -
INFO:__main__:Loading model pipeline_demo_best_v2...


Loaded: Pipeline(memory=None,
     steps=[('prep', PrepPipeline(add_missing_indicators=True, copy=True, impute_age=True,
       impute_cabin=True, notes='v2-default pipeline', train_filter='')), ('featurize', DataFrameMapper(default=False, df_out=True,
        features=[(['PassengerId'], Imputer(axis=0, copy=True, missing_values='NaN...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])


INFO:__main__:application/json


reading json...
[{'PassengerId': 892, 'Pclass': 3, 'Name': 'Kelly, Mr. James', 'Sex': 'male', 'Age': 34.5, 'SibSp': 0, 'Parch': 0, 'Ticket': '330911', 'Fare': 7.8292, 'Cabin': None, 'Embarked': 'Q'}, {'PassengerId': 893, 'Pclass': 3, 'Name': 'Wilkes, Mrs. James (Ellen Needs)', 'Sex': 'female', 'Age': 47.0, 'SibSp': 1, 'Parch': 0, 'Ticket': '363272', 'Fare': 7.0, 'Cabin': None, 'Embarked': 'S'}, {'PassengerId': 894, 'Pclass': 2, 'Name': 'Myles, Mr. Thomas Francis', 'Sex': 'male', 'Age': 62.0, 'SibSp': 0, 'Parch': 0, 'Ticket': '240276', 'Fare': 9.6875, 'Cabin': None, 'Embarked': 'Q'}, {'PassengerId': 895, 'Pclass': 3, 'Name': 'Wirz, Mr. Albert', 'Sex': 'male', 'Age': 27.0, 'SibSp': 0, 'Parch': 0, 'Ticket': '315154', 'Fare': 8.6625, 'Cabin': None, 'Embarked': 'S'}, {'PassengerId': 896, 'Pclass': 3, 'Name': 'Hirvonen, Mrs. Alexander (Helga E Lindqvist)', 'Sex': 'female', 'Age': 22.0, 'SibSp': 1, 'Parch': 1, 'Ticket': '3101298', 'Fare': 12.2875, 'Cabin': None, 'Embarked': 'S'}]
json end...


INFO:werkzeug:127.0.0.1 - - [12/Sep/2018 16:50:58] "POST /predict/pipeline_demo_best_v2/ HTTP/1.1" 200 -
